# ML Preprocessing for Real Estate Southern Spain

This notebook is for machine learning preprocessing tasks.

In [1]:
import numpy as np
import pandas as pd
from collections import Counter
from sklearn.model_selection import train_test_split, KFold
from sklearn.preprocessing import OneHotEncoder


In [2]:
# Load data
# Add your preprocessing code here

In [3]:
df = pd.read_csv('../data/properties.csv')
pd.set_option('display.float_format', '{:,.0f}'.format)
#pd.reset_option('display.float_format')
df_real = df.copy()
df.columns = df.columns.str.strip()

In [4]:
#df.head()#df.columns
ref = df['reference'].tolist()
df['reference'].unique()

array(['R126116', 'R132516', 'R121563', ..., 'R90252', 'R89424',
       'R706006'], dtype=object)

In [5]:
df['price'] = (
    df['price']
    .astype(str)
    .str.replace(r'[^\d.,]', '', regex=True) 
    .str.replace(',', '')                     
    .astype(float)
)

df['features'] = (
    df['pipe-separed list of features of the property']
    .str.lower()
    .str.split('|')
)

df['property_type'] = (
    df['title']
    .str.replace(r'^\d+\s*Bedroom(s)?\s*', '', regex=True)
    .str.replace(r'^\d+\s*', '', regex=True)    
    .str.strip()
)

print(df['property_type'].unique())

df['y'] = np.log(df['price'])
df.describe()

['Apartment' 'Other' 'Finca' 'Plot' 'Detached Villa' 'Semi-detached'
 'Penthouse' 'Town House' 'Detached' 'Warehouse' 'Commercial Property'
 'Land' 'Guest House' 'Hospitality' 'Commercial Development' 'Shop'
 'Property' 'Bar / Nightclub' 'Equestrian Facility' 'Restaurant' 'Office'
 'Sheltered Housing' 'Bungalow' 'House' 'Farm' 'Ground Flat' 'Parking'
 'Garage' 'Cafe' 'Storage' 'Studio' 'Duplex' 'Cave House']


,price,bedrooms,bathrooms,indoor surface area in sqm,outdoor surface area in sqm,y
count,"12,086","12,084","10,952","10,414","3,821","12,086"
mean,"1,046,977","2,534","1,076",233,"11,132",13
std,"1,881,298","98,531","46,365",265,"196,065",1
min,"4,500",0,0,10,1,8
25%,"285,000",2,2,100,401,13
50%,"475,000",3,2,144,"1,009",13
75%,"995,000",5,4,269,"2,342",14
max,"50,000,000","9,700,000","4,630,000","5,367","9,350,000",18


In [6]:
non_residential = {
    'Plot', 'Land', 'Commercial Property', 'Commercial Development',
    'Warehouse', 'Shop', 'Office', 'Restaurant', 'Bar / Nightclub',
    'Cafe', 'Parking', 'Garage', 'Storage', 'Property', 'Other'
}

df.loc[df['property_type'].isin(non_residential), 'bedrooms'] = np.nan
df.loc[df['property_type'].isin(non_residential), 'bathrooms'] = np.nan
df.head()

,reference,location,price,title,bedrooms,bathrooms,indoor surface area in sqm,outdoor surface area in sqm,pipe-separed list of features of the property,features,property_type,y
0,R126116,"Nueva Andalucía, Costa del Sol","450,000",3 Bedroom Apartment,3,2,120,NaN,Close to Shops|Electricity Utilities|Close to ...,"[close to shops, electricity utilities, close ...",Apartment,13
1,R132516,"Puerto Banús, Costa del Sol","1,250,000",Other,NaN,NaN,NaN,NaN,Electricity Utilities|Close to Town|Street Vie...,"[electricity utilities, close to town, street ...",Other,14
2,R121563,"Antequera, Costa del Sol","749,000",12 Bedroom Finca,12,7,366,"5,353",Mountain Views|Private Garden|Open Parking|Urb...,"[mountain views, private garden, open parking,...",Finca,14
3,R127293,"La Mairena, Costa del Sol","500,000",Plot,NaN,NaN,NaN,NaN,Mountain Views|Private Garden|Electricity Util...,"[mountain views, private garden, electricity u...",Plot,13
4,R114041,"Benalmadena, Costa del Sol","1,695,000",4 Bedroom Detached Villa,4,4,634,"1,000",Private Garden|Electricity Utilities|Drinkable...,"[private garden, electricity utilities, drinka...",Detached Villa,14


In [7]:
df['location'] = df['location'].str.strip()
df[['city', 'region']] = df['location'].str.split(',', n=1, expand=True)
df['city'] = df['city'].str.strip()
df['region'] = df['region'].str.strip()

df['city'].nunique()

226

In [8]:
train_df, test_df = train_test_split(
    df,
    test_size=0.2,
    random_state=42
)


In [9]:
tr, val = train_test_split(
    train_df,
    test_size=0.2,
    random_state=42
)

limit = {}
for col in ['price', 'indoor surface area in sqm', 'outdoor surface area in sqm']:
    limit[col] = tr[col].quantile(0.995)

limit['bedrooms'] = tr['bedrooms'].quantile(0.99)
limit['bathrooms'] = tr['bathrooms'].quantile(0.99)

def apply_caps(df, limit):
    df = df.copy()
    for col, cap in limit.items():
        df[col] = df[col].clip(upper=cap)
    return df

tr = apply_caps(tr, limit)
val = apply_caps(val, limit)
test_df = apply_caps(test_df, limit)


In [10]:
global_mean = tr['y'].mean()

city_stats = (
    tr.groupby('city')['y']
    .agg(['mean', 'count'])
)

k = 20
city_stats['mean'] = (
    (city_stats['count'] * city_stats['mean'] + k * global_mean) /
    (city_stats['count'] + k)
)

def add_city_mean(df):
    return df.assign(
        city_mean=df['city'].map(city_stats['mean']).fillna(global_mean)
    )

tr = add_city_mean(tr)
val = add_city_mean(val)
test_df = add_city_mean(test_df)


In [11]:
from sklearn.preprocessing import OneHotEncoder

ohe = OneHotEncoder(handle_unknown='ignore', sparse_output=False)
ohe.fit(tr[['property_type']])

ohe_cols = ohe.get_feature_names_out(['property_type'])

def apply_ohe(df):
    ohe_arr = ohe.transform(df[['property_type']])
    df[ohe_cols] = ohe_arr
    return df

tr = apply_ohe(tr)
val = apply_ohe(val)
test_df = apply_ohe(test_df)


In [12]:
#df['features'] = df['pipe-separed list of features of the property'].str.split('|')

counts = Counter(tr["features"].explode().str.strip())
top_10 = counts.most_common(30)
top_10.append(("Other", 1))
d = dict(top_10)
names = list(d.keys())
names

['fully fitted kitchen',
 'private terrace',
 'air conditioning climate control',
 'fitted wardrobes',
 'close to shops',
 'covered terrace',
 'electricity utilities',
 'close to schools',
 'excellent condition',
 'communal pool',
 'south orientation',
 'sea views',
 'mountain views',
 'urbanisation',
 'ensuite bathroom',
 'close to sea',
 'communal garden',
 'gated complex security',
 'near transport',
 'garden views',
 'double glazing',
 'drinkable water utilities',
 'close to town',
 'storage room',
 'resale',
 'lift',
 'close to golf',
 'investment',
 'panoramic views',
 'private parking',
 'Other']

In [13]:
def add_feature_cols(df, features):
    for f in features:
        df[f] = df['features'].apply(
            lambda x: int(f in x) if isinstance(x, str) else 0
        )
    return df

tr = add_feature_cols(tr, names)
val = add_feature_cols(val, names)
test_df = add_feature_cols(test_df, names)

In [14]:
tr['price_log'] = np.log1p(tr['price'])
val['price_log'] = np.log1p(val['price'])

for col in ['indoor surface area in sqm', 'outdoor surface area in sqm']:
    tr[f'{col}_log'] = np.log1p(tr[col])
    val[f'{col}_log'] = np.log1p(val[col])
    test_df[f'{col}_log'] = np.log1p(test_df[col])

y_tr = tr['price_log']
y_val = val['price_log']

X_tr = tr.drop(columns=['price', 'price_log'])
X_val = val.drop(columns=['price', 'price_log'])


In [15]:
feature_cols = (['city_mean', 'bedrooms', 'bathrooms','indoor surface area in sqm',
     'outdoor surface area in sqm'] + names + list(ohe_cols))

X_tr = tr[feature_cols]
X_val = val[feature_cols]
X_test = test_df[feature_cols]


In [16]:
X_tr.info()
print()
X_tr.tail()
#for col in df.select_dtypes(include=['object']).columns:
#    print(f"Unique values in {col}: {df[col].nunique()}")




<class 'pandas.core.frame.DataFrame'>
Index: 7734 entries, 9102 to 9047
Data columns (total 68 columns):
 #   Column                                Non-Null Count  Dtype  
---  ------                                --------------  -----  
 0   city_mean                             7734 non-null   float64
 1   bedrooms                              6694 non-null   float64
 2   bathrooms                             6681 non-null   float64
 3   indoor surface area in sqm            6641 non-null   float64
 4   outdoor surface area in sqm           2429 non-null   float64
 5   fully fitted kitchen                  7734 non-null   int64  
 6   private terrace                       7734 non-null   int64  
 7   air conditioning climate control      7734 non-null   int64  
 8   fitted wardrobes                      7734 non-null   int64  
 9   close to shops                        7734 non-null   int64  
 10  covered terrace                       7734 non-null   int64  
 11  electricity utiliti

,city_mean,bedrooms,bathrooms,indoor surface area in sqm,outdoor surface area in sqm,fully fitted kitchen,private terrace,air conditioning climate control,fitted wardrobes,close to shops,...,property_type_Plot,property_type_Property,property_type_Restaurant,property_type_Semi-detached,property_type_Sheltered Housing,property_type_Shop,property_type_Storage,property_type_Studio,property_type_Town House,property_type_Warehouse
9913,13,2,2,94,NaN,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3875,13,3,3,189,NaN,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2934,13,3,2,145,NaN,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
6759,14,6,7,987,"1,871",0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9047,13,2,2,168,NaN,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [17]:
X_tr.columns

Index(['city_mean', 'bedrooms', 'bathrooms', 'indoor surface area in sqm',
       'outdoor surface area in sqm', 'fully fitted kitchen',
       'private terrace', 'air conditioning climate control',
       'fitted wardrobes', 'close to shops', 'covered terrace',
       'electricity utilities', 'close to schools', 'excellent condition',
       'communal pool', 'south orientation', 'sea views', 'mountain views',
       'urbanisation', 'ensuite bathroom', 'close to sea', 'communal garden',
       'gated complex security', 'near transport', 'garden views',
       'double glazing', 'drinkable water utilities', 'close to town',
       'storage room', 'resale', 'lift', 'close to golf', 'investment',
       'panoramic views', 'private parking', 'Other',
       'property_type_Apartment', 'property_type_Bar / Nightclub',
       'property_type_Bungalow', 'property_type_Cafe',
       'property_type_Commercial Development',
       'property_type_Commercial Property', 'property_type_Detached',
     

In [18]:
import sys
print(sys.path)


['/Users/linas.danusevicius/Desktop/University/4_Machine Learning Algorithms 1/IP', '/Applications/PyCharm.app/Contents/plugins/python-ce/helpers/pydev', '/Applications/PyCharm.app/Contents/plugins/python-ce/helpers/jupyter_debug', '/Library/Developer/CommandLineTools/Library/Frameworks/Python3.framework/Versions/3.9/lib/python39.zip', '/Library/Developer/CommandLineTools/Library/Frameworks/Python3.framework/Versions/3.9/lib/python3.9', '/Library/Developer/CommandLineTools/Library/Frameworks/Python3.framework/Versions/3.9/lib/python3.9/lib-dynload', '', '/Users/linas.danusevicius/Library/Python/3.9/lib/python/site-packages', '/Library/Developer/CommandLineTools/Library/Frameworks/Python3.framework/Versions/3.9/lib/python3.9/site-packages']


In [19]:
import preproc


/Users/linas.danusevicius/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


LOADED preproc.py
cpu


In [20]:
import preproc

preproc.encode_descriptions()

Batches:   0%|          | 0/378 [00:00<?, ?it/s]

(      reference                        location       price  \
 0       R126116  Nueva Andalucía, Costa del Sol    €450,000   
 1       R132516     Puerto Banús, Costa del Sol  €1,250,000   
 2       R121563        Antequera, Costa del Sol    €749,000   
 3       R127293       La Mairena, Costa del Sol    €500,000   
 4       R114041      Benalmadena, Costa del Sol  €1,695,000   
 ...         ...                             ...         ...   
 12081    R89338      Mijas Costa, Costa del Sol  €3,500,000   
 12082   R823240        Costalita, Costa del Sol  €1,400,000   
 12083    R90252  Guadalmina Alta, Costa del Sol  €1,015,000   
 12084    R89424        La Quinta, Costa del Sol    €395,000   
 12085   R706006         Cabopino, Costa del Sol  €1,750,000   
 
                           title   bedrooms   bathrooms  \
 0           3 Bedroom Apartment          3           2   
 1                         Other          1         504   
 2              12 Bedroom Finca         12          

In [21]:
preproc.encode_images()


Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.
Encoding images: 100%|██████████| 12086/12086 [33:35<00:00,  6.00it/s] 


(      reference                        location       price  \
 0       R126116  Nueva Andalucía, Costa del Sol    €450,000   
 1       R132516     Puerto Banús, Costa del Sol  €1,250,000   
 2       R121563        Antequera, Costa del Sol    €749,000   
 3       R127293       La Mairena, Costa del Sol    €500,000   
 4       R114041      Benalmadena, Costa del Sol  €1,695,000   
 ...         ...                             ...         ...   
 12081    R89338      Mijas Costa, Costa del Sol  €3,500,000   
 12082   R823240        Costalita, Costa del Sol  €1,400,000   
 12083    R90252  Guadalmina Alta, Costa del Sol  €1,015,000   
 12084    R89424        La Quinta, Costa del Sol    €395,000   
 12085   R706006         Cabopino, Costa del Sol  €1,750,000   
 
                           title   bedrooms   bathrooms  \
 0           3 Bedroom Apartment          3           2   
 1                         Other          1         504   
 2              12 Bedroom Finca         12          

In [22]:
X_tr, y_tr, X_val, y_val, X_test, feature_cols = preproc.get_preprocessed_data()
print(X_tr.shape, y_tr.shape, X_val.shape, y_val.shape, X_test.shape)
print(feature_cols[:10])

/Users/linas.danusevicius/Desktop/University/4_Machine Learning Algorithms 1/IP/Real-Estate-Southern-Spain-2024/models/preproc.py:126: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_[cols] = emb
/Users/linas.danusevicius/Desktop/University/4_Machine Learning Algorithms 1/IP/Real-Estate-Southern-Spain-2024/models/preproc.py:126: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_[cols] = emb
/Users/linas.danusevicius/Desktop/University/4_Machine Learning Algorithms 1/IP/Real-Estate-Southern-Spain-2024/models/preproc.py:126: Perfo

(7734, 1345) (7734,) (1934, 1345) (1934,) (2418, 1345)
['city_mean', 'bedrooms', 'bathrooms', 'fully fitted kitchen', 'private terrace', 'air conditioning climate control', 'fitted wardrobes', 'close to shops', 'covered terrace', 'electricity utilities']
